[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ruliana/pytorch-katas/blob/main/dan_1/kata_02_temple_door_humidity_predictor_unrevised.ipynb)

## 🏮 The Ancient Scroll Unfurls 🏮

**COOK OH-PAI-TIMIZER'S DOOR WISDOM: THE SIGMOID GATE MYSTERY**

Dan Level: 1 (Temple Sweeper) | Time: 45 minutes | Sacred Arts: Binary Classification, Sigmoid Activation, Threshold Decisions

## 📜 THE CHALLENGE

Cook Oh-Pai-Timizer bustles through the temple corridors, balancing steaming bowls of sacred soup and muttering about the ancient wooden doors. "These old doors," Cook says, wiping sweat from their brow, "they have minds of their own! When the humidity rises, some stick like they're guarded by stubborn spirits, while others swing freely. Yesterday I nearly spilled an entire pot of precious lotus root broth trying to push through the meditation hall door!"

The wise cook has been observing patterns for months, noting how the temple's humidity affects each door's behavior. Now Cook seeks to master the art of prediction—to know which doors will stick before approaching them with precious cargo. "If I can learn this pattern," Cook explains, "I can plan my routes and avoid the sticky doors entirely. But I need more than just intuition—I need the mathematical wisdom of the sigmoid function to transform humidity measurements into clear yes-or-no decisions!"

## 🎯 THE SACRED OBJECTIVES

By the end of this kata, you will have mastered:

- [ ] **Binary Classification Fundamentals**: Learn to predict yes/no outcomes using neural networks
- [ ] **Sigmoid Activation Mastery**: Transform continuous values into probabilities between 0 and 1
- [ ] **Single-Variable Classification**: Build intuition with one input feature before tackling complex problems
- [ ] **Threshold Decision Making**: Convert probabilities into actionable binary decisions
- [ ] **Binary Cross-Entropy Loss**: Understand the mathematics of classification error measurement
- [ ] **Probability Interpretation**: Learn to read and trust your model's confidence levels

In [ ]:
# 📦 FIRST CELL - ALL IMPORTS AND CONFIGURATION
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from typing import Tuple

# Set reproducibility
torch.manual_seed(42)

# Global configuration constants
DEFAULT_CHAOS_LEVEL = 0.1
SACRED_SEED = 42
HUMIDITY_THRESHOLD = 60.0  # Cook's observed critical humidity level

## 🍜 THE SACRED DATA GENERATION SCROLL

In [ ]:
def generate_door_humidity_data(n_observations: int = 200, chaos_level: float = 0.1, 
                               sacred_seed: int = 42) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Generate observations of temple door behavior based on humidity levels.
    
    Cook Oh-Pai-Timizer's wisdom: Doors tend to stick when humidity > 60%
    But ancient wood has its own mysterious patterns!
    
    Args:
        n_observations: Number of door-testing incidents to simulate
        chaos_level: Amount of wooden unpredictability (0.0 = perfectly predictable doors, 1.0 = chaos)
        sacred_seed: Ensures consistent randomness for reproducible soup deliveries
        
    Returns:
        Tuple of (humidity_levels, door_sticks) as sacred tensors
        door_sticks: 1 = door sticks (avoid!), 0 = door opens smoothly
    """
    torch.manual_seed(sacred_seed)
    
    # Temple humidity ranges from 20% (dry winter) to 90% (monsoon season)
    humidity_levels = torch.rand(n_observations, 1) * 70 + 20
    
    # Cook's observed pattern: doors stick more often when humidity > 60%
    # Create base probability using a smooth sigmoid-like pattern
    stick_probability = torch.sigmoid((humidity_levels.squeeze() - HUMIDITY_THRESHOLD) / 5.0)
    
    # Add wooden chaos - sometimes doors surprise you!
    chaos = torch.randn(n_observations) * chaos_level * 0.3
    stick_probability = torch.clamp(stick_probability + chaos, 0.0, 1.0)
    
    # Convert probabilities to actual door behavior (0 or 1)
    door_sticks = torch.bernoulli(stick_probability).unsqueeze(1)
    
    return humidity_levels, door_sticks

def visualize_door_wisdom(humidity: torch.Tensor, door_sticks: torch.Tensor, 
                         predictions: torch.Tensor = None):
    """
    Display the sacred patterns of door behavior vs humidity.
    """
    plt.figure(figsize=(14, 8))
    
    # Create two subplots
    plt.subplot(1, 2, 1)
    
    # Separate sticky and smooth doors for clear visualization
    sticky_mask = door_sticks.squeeze() == 1
    smooth_mask = door_sticks.squeeze() == 0
    
    plt.scatter(humidity[sticky_mask].numpy(), [1]*torch.sum(sticky_mask).item(), 
                alpha=0.6, color='red', s=50, label='Doors That Stick (Danger!)')
    plt.scatter(humidity[smooth_mask].numpy(), [0]*torch.sum(smooth_mask).item(), 
                alpha=0.6, color='green', s=50, label='Doors That Open Smoothly')
    
    plt.axvline(x=HUMIDITY_THRESHOLD, color='orange', linestyle='--', alpha=0.7,
                label=f'Cook\'s Threshold ({HUMIDITY_THRESHOLD}% humidity)')
    
    plt.xlabel('Humidity Level (%)')
    plt.ylabel('Door Behavior')
    plt.title('Cook Oh-Pai-Timizer\'s Door Observations')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.ylim(-0.2, 1.2)
    plt.yticks([0, 1], ['Opens Smoothly', 'Sticks!'])
    
    # Second subplot for predictions if provided
    if predictions is not None:
        plt.subplot(1, 2, 2)
        
        # Sort by humidity for smooth prediction curve
        sorted_indices = torch.argsort(humidity.squeeze())
        sorted_humidity = humidity[sorted_indices]
        sorted_predictions = predictions[sorted_indices]
        
        plt.plot(sorted_humidity.numpy(), sorted_predictions.detach().numpy(), 
                'gold', linewidth=3, label='Your Sigmoid Predictions')
        plt.scatter(humidity[sticky_mask].numpy(), [1]*torch.sum(sticky_mask).item(), 
                    alpha=0.4, color='red', s=30)
        plt.scatter(humidity[smooth_mask].numpy(), [0]*torch.sum(smooth_mask).item(), 
                    alpha=0.4, color='green', s=30)
        
        plt.axhline(y=0.5, color='purple', linestyle=':', alpha=0.7,
                    label='Decision Threshold (50%)')
        plt.axvline(x=HUMIDITY_THRESHOLD, color='orange', linestyle='--', alpha=0.7)
        
        plt.xlabel('Humidity Level (%)')
        plt.ylabel('Predicted Probability of Sticking')
        plt.title('Your Mystical Door Predictions')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.ylim(-0.1, 1.1)
    
    plt.tight_layout()
    plt.show()

## 🚪 THE SIGMOID GATEWAY PREDICTOR

In [ ]:
class DoorStickinessPredictor(nn.Module):
    """A mystical artifact for predicting when temple doors will misbehave."""
    
    def __init__(self, input_features: int = 1):
        super(DoorStickinessPredictor, self).__init__()
        # TODO: Create a Linear layer to transform humidity into raw predictions
        # Hint: One input (humidity), one output (raw stickiness score)
        self.linear = None
        
        # TODO: Add the sigmoid activation function
        # Hint: torch.nn.Sigmoid() transforms any number into a probability (0 to 1)
        self.sigmoid = None
    
    def forward(self, features: torch.Tensor) -> torch.Tensor:
        """Transform humidity measurements into door-sticking probabilities."""
        # TODO: Pass humidity through the linear layer first
        raw_output = None
        
        # TODO: Apply sigmoid to convert raw output to probability (0-1 range)
        # This is the magic that makes classification work!
        probability = None
        
        return probability

def train_door_predictor(model: nn.Module, features: torch.Tensor, target: torch.Tensor,
                        epochs: int = 2000, learning_rate: float = 0.1) -> list:
    """
    Train the door stickiness prediction model.
    
    Returns:
        List of loss values during training
    """
    # TODO: Choose the right loss function for binary classification
    # Hint: Binary Cross Entropy Loss is the master's choice for yes/no problems
    criterion = None
    
    # TODO: Choose your optimizer
    # Hint: SGD with higher learning rate works well for simple problems
    optimizer = None
    
    losses = []
    
    for epoch in range(epochs):
        # TODO: CRITICAL - Clear gradients from previous iteration
        # The spirits of old gradients must be banished!
        
        # TODO: Forward pass - get probability predictions
        predictions = None
        
        # TODO: Calculate binary classification loss
        loss = None
        
        # TODO: Backward pass - compute gradients
        
        # TODO: Update model parameters
        
        losses.append(loss.item())
        
        # Report progress to Cook Oh-Pai-Timizer
        if (epoch + 1) % 200 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
            if loss.item() < 0.3:
                print("🍜 Cook Oh-Pai-Timizer nods approvingly - the wisdom flows!")
    
    return losses

def make_door_decisions(model: nn.Module, humidity: torch.Tensor, threshold: float = 0.5) -> torch.Tensor:
    """
    Convert probability predictions into binary door decisions.
    
    Args:
        model: Your trained predictor
        humidity: Humidity measurements
        threshold: Decision boundary (default 0.5 means 50% confidence)
    
    Returns:
        Binary decisions: 1 = door will stick (avoid!), 0 = door opens smoothly
    """
    with torch.no_grad():
        probabilities = model(humidity)
        decisions = (probabilities > threshold).float()
    return decisions

## ⚡ THE TRIALS OF MASTERY

### Trial 1: Basic Sigmoid Mastery
- [ ] Loss decreases smoothly (no oscillating spirits)
- [ ] Final loss below 0.4 (Cook Oh-Pai-Timizer's approval threshold)
- [ ] Model outputs probabilities between 0 and 1 (sigmoid magic working)
- [ ] Predictions show clear S-curve pattern when plotted against humidity

### Trial 2: Decision Accuracy Test
- [ ] Achieve >75% accuracy on training data
- [ ] Model correctly identifies most doors above 60% humidity as "sticky"
- [ ] Model correctly identifies most doors below 60% humidity as "smooth"

### Trial 3: Understanding Test

In [ ]:
def test_your_wisdom(model):
    """Cook Oh-Pai-Timizer's evaluation of your door prediction skills."""
    
    # Test sigmoid activation is working
    test_humidity = torch.tensor([[30.0], [60.0], [80.0]])  # Low, medium, high humidity
    predictions = model(test_humidity)
    
    # All predictions should be probabilities (0 to 1)
    assert torch.all(predictions >= 0) and torch.all(predictions <= 1), \
        "Sigmoid not working - predictions outside 0-1 range!"
    
    # Higher humidity should generally mean higher sticking probability
    assert predictions[2] > predictions[0], \
        "High humidity should be stickier than low humidity!"
    
    # Check model learned reasonable threshold behavior
    low_humidity_pred = model(torch.tensor([[40.0]])).item()
    high_humidity_pred = model(torch.tensor([[75.0]])).item()
    
    assert low_humidity_pred < 0.5, f"Low humidity prediction {low_humidity_pred:.3f} should be < 0.5"
    assert high_humidity_pred > 0.5, f"High humidity prediction {high_humidity_pred:.3f} should be > 0.5"
    
    print("🎉 Cook Oh-Pai-Timizer beams with pride!")
    print("   'You have mastered the sigmoid way - doors shall no longer surprise you!'")
    print(f"   Low humidity (40%): {low_humidity_pred:.1%} chance of sticking")
    print(f"   High humidity (75%): {high_humidity_pred:.1%} chance of sticking")

## 🌸 THE FOUR PATHS OF MASTERY: PROGRESSIVE EXTENSIONS

### Extension 1: Master Pai-Torch's Confidence Intervals
*"The wise cook knows not just whether a door will stick, but how certain that knowledge is."*

*Master Pai-Torch appears in a swirl of steam from the kitchen*

*"Grasshopper, I observe your binary wisdom grows strong. But true mastery lies not just in prediction, but in understanding the confidence of that prediction. When your model says '70% chance of sticking,' what does this truly mean for Cook's soup delivery route?"*

**NEW CONCEPTS:** Probability interpretation, confidence thresholds, decision risk analysis  
**DIFFICULTY:** +15% (still Dan 1, but deeper understanding)

In [ ]:
def analyze_prediction_confidence(model, humidity, target, confidence_thresholds=[0.3, 0.5, 0.7, 0.9]):
    """
    Analyze how confident predictions perform with different decision thresholds.
    
    Args:
        confidence_thresholds: Different cutoff points for "door will stick" decisions
    
    Returns:
        Dictionary of {threshold: (accuracy, precision, recall)}
    """
    results = {}
    
    with torch.no_grad():
        probabilities = model(humidity)
        
        for threshold in confidence_thresholds:
            # TODO: Convert probabilities to binary decisions using this threshold
            decisions = None
            
            # TODO: Calculate accuracy (correct predictions / total predictions)
            accuracy = None
            
            # TODO: Calculate precision (correct "stick" predictions / all "stick" predictions)
            # When you predict "door will stick," how often are you right?
            precision = None
            
            # TODO: Calculate recall (correct "stick" predictions / all actual sticky doors)
            # Of all doors that actually stick, how many did you catch?
            recall = None
            
            results[threshold] = (accuracy, precision, recall)
            
    return results

def visualize_confidence_wisdom(results):
    """Display how different confidence thresholds affect decision quality."""
    thresholds = list(results.keys())
    accuracies = [results[t][0] for t in thresholds]
    precisions = [results[t][1] for t in thresholds]
    recalls = [results[t][2] for t in thresholds]
    
    plt.figure(figsize=(10, 6))
    plt.plot(thresholds, accuracies, 'o-', label='Accuracy', linewidth=2)
    plt.plot(thresholds, precisions, 's-', label='Precision', linewidth=2)
    plt.plot(thresholds, recalls, '^-', label='Recall', linewidth=2)
    
    plt.xlabel('Confidence Threshold')
    plt.ylabel('Performance Score')
    plt.title('Master Pai-Torch\'s Confidence Analysis')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.ylim(0, 1.1)
    plt.show()

# TRIAL: Find the optimal confidence threshold for Cook's soup delivery needs
# SUCCESS: Understand the tradeoff between catching sticky doors vs. false alarms
# MASTERY: Explain why Cook might prefer high recall over high precision

### Extension 2: He-Ao-World's Measurement Mishaps
*"Oh dear! My old hands mixed up some of the humidity readings..."*

*He-Ao-World shuffles over, looking apologetic as always*

*"I was maintaining the humidity sensors yesterday and, well... some of them might be reading in different units now. Some show relative humidity, others show absolute humidity, and a few might be completely miscalibrated. The data looks quite scattered now. Cook Oh-Pai-Timizer is most understanding, but we need to make sense of this chaos before the evening soup service!"*

**NEW CONCEPTS:** Data preprocessing, outlier detection, robust training  
**DIFFICULTY:** +25% (still Dan 1, but messier real-world data)

In [ ]:
def generate_messy_humidity_data(n_observations: int = 200, chaos_level: float = 0.2):
    """
    Generate humidity data with He-Ao-World's "accidental" measurement inconsistencies.
    
    Returns:
        Messy humidity data with different scales and outliers
    """
    # Start with clean data
    clean_humidity, door_sticks = generate_door_humidity_data(n_observations, chaos_level)
    
    # He-Ao-World's "accidents"
    messy_humidity = clean_humidity.clone()
    
    # 20% of readings are in absolute humidity (different scale)
    absolute_mask = torch.rand(n_observations) < 0.2
    messy_humidity[absolute_mask] = messy_humidity[absolute_mask] * 2.5 + 30
    
    # 10% of readings are completely wrong (sensor failures)
    broken_mask = torch.rand(n_observations) < 0.1
    messy_humidity[broken_mask] = torch.rand(torch.sum(broken_mask).item(), 1) * 200 + 100
    
    # 5% of readings are negative (impossible but happens with broken sensors)
    negative_mask = torch.rand(n_observations) < 0.05
    messy_humidity[negative_mask] = -torch.rand(torch.sum(negative_mask).item(), 1) * 50
    
    return messy_humidity, door_sticks

def clean_humidity_data(messy_humidity: torch.Tensor) -> torch.Tensor:
    """
    Clean He-Ao-World's messy humidity measurements.
    
    Returns:
        Cleaned humidity data ready for training
    """
    cleaned = messy_humidity.clone()
    
    # TODO: Remove impossible values (negative humidity, >100% relative humidity)
    # Hint: Use torch.clamp to constrain values to reasonable range
    
    # TODO: Detect and handle outliers (values way outside normal range)
    # Hint: Calculate mean and standard deviation, cap extreme values
    
    # TODO: Normalize the data to 0-100 range for consistency
    # Hint: min-max normalization works well here
    
    return cleaned

def visualize_data_cleaning(messy_data, clean_data, target):
    """Show the before and after of He-Ao-World's data cleaning."""
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.scatter(messy_data.numpy(), target.numpy(), alpha=0.6, color='red')
    plt.xlabel('Messy Humidity Readings')
    plt.ylabel('Door Sticks (1=Yes, 0=No)')
    plt.title('He-Ao-World\'s "Oops!"')
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 3, 2)
    plt.scatter(clean_data.numpy(), target.numpy(), alpha=0.6, color='blue')
    plt.xlabel('Cleaned Humidity Readings')
    plt.ylabel('Door Sticks (1=Yes, 0=No)')
    plt.title('After Cleaning')
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 3, 3)
    plt.hist(messy_data.numpy(), bins=30, alpha=0.5, color='red', label='Messy')
    plt.hist(clean_data.numpy(), bins=30, alpha=0.5, color='blue', label='Clean')
    plt.xlabel('Humidity Values')
    plt.ylabel('Frequency')
    plt.title('Data Distribution')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# TRIAL: Train your model on both messy and clean data
# SUCCESS: Clean data achieves significantly better performance
# MASTERY: Understand why data quality matters more than model complexity

### Extension 3: Cook Oh-Pai-Timizer's Route Optimization
*"A wise cook plans the path, not just the destination!"*

*Cook Oh-Pai-Timizer sets down a wooden spoon and unfolds an ancient map of the temple*

*"Now that you can predict individual doors, young grasshopper, let us think bigger! I must deliver soup to five different halls, and each has multiple entrances. Your door wisdom must guide me to find the path of least resistance - the route where I'm most likely to encounter only smooth-opening doors. This is optimization, my eager student!"*

**NEW CONCEPTS:** Multi-door prediction, path optimization, practical decision making  
**DIFFICULTY:** +35% (still Dan 1, but applying knowledge to complex scenarios)

In [ ]:
def generate_temple_route_data(n_routes: int = 50):
    """
    Generate data for multiple temple routes, each with several doors.
    
    Returns:
        Dictionary with route information and door predictions needed
    """
    routes = {}
    
    for route_id in range(n_routes):
        # Each route has 3-7 doors to pass through
        n_doors = torch.randint(3, 8, (1,)).item()
        
        # Generate humidity for each door along this route
        door_humidity = torch.rand(n_doors, 1) * 70 + 20
        
        routes[f'route_{route_id}'] = {
            'door_humidity': door_humidity,
            'n_doors': n_doors
        }
    
    return routes

def predict_route_success(model, route_humidity: torch.Tensor, success_threshold: float = 0.8):
    """
    Predict the probability that Cook can complete a route without encountering sticky doors.
    
    Args:
        route_humidity: Humidity levels for all doors on this route
        success_threshold: What probability of "smooth" door is considered safe
    
    Returns:
        Overall route success probability
    """
    with torch.no_grad():
        # TODO: Get sticking probabilities for each door
        stick_probabilities = None
        
        # TODO: Convert to "smooth opening" probabilities
        smooth_probabilities = None
        
        # TODO: Calculate overall route success probability
        # Hint: All doors must open smoothly, so multiply probabilities
        route_success_prob = None
        
    return route_success_prob

def find_best_routes(model, routes_data, top_k: int = 5):
    """
    Find the safest routes for Cook's soup delivery mission.
    
    Returns:
        List of (route_id, success_probability) sorted by safety
    """
    route_scores = []
    
    for route_id, route_info in routes_data.items():
        # TODO: Calculate success probability for this route
        success_prob = None
        
        route_scores.append((route_id, success_prob.item()))
    
    # Sort by success probability (highest first)
    route_scores.sort(key=lambda x: x[1], reverse=True)
    
    return route_scores[:top_k]

def visualize_route_analysis(model, routes_data, best_routes):
    """Display Cook's route optimization wisdom."""
    plt.figure(figsize=(15, 10))
    
    # Show top 3 routes
    for i, (route_id, success_prob) in enumerate(best_routes[:3]):
        plt.subplot(2, 3, i+1)
        
        route_info = routes_data[route_id]
        humidity = route_info['door_humidity']
        
        with torch.no_grad():
            stick_probs = model(humidity)
        
        door_numbers = range(1, len(humidity) + 1)
        plt.bar(door_numbers, stick_probs.numpy().flatten(), 
                color=['green' if p < 0.5 else 'red' for p in stick_probs.flatten()])
        
        plt.title(f'Route {i+1}\nSuccess: {success_prob:.1%}')
        plt.xlabel('Door Number')
        plt.ylabel('Stick Probability')
        plt.ylim(0, 1)
        plt.axhline(y=0.5, color='purple', linestyle='--', alpha=0.7)
    
    # Overall route success distribution
    plt.subplot(2, 1, 2)
    all_success_probs = [success_prob for _, success_prob in 
                        [(rid, predict_route_success(model, routes_data[rid]['door_humidity']).item()) 
                         for rid in routes_data.keys()]]
    
    plt.hist(all_success_probs, bins=20, alpha=0.7, color='blue')
    plt.axvline(x=np.mean(all_success_probs), color='red', linestyle='--', 
                label=f'Average: {np.mean(all_success_probs):.1%}')
    plt.xlabel('Route Success Probability')
    plt.ylabel('Number of Routes')
    plt.title('Distribution of All Route Success Rates')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# TRIAL: Find the safest routes for Cook's soup delivery
# SUCCESS: Identify routes with >80% success probability
# MASTERY: Understand how individual predictions combine into system-level decisions

### Extension 4: Suki's Seasonal Wisdom
*"Meow meow purr... meow purr purr meow." (Translation: "Young human, the doors behave differently in different seasons.")*

*Suki sits regally beside a weather scroll, tail twitching with knowing wisdom*

*Master Pai-Torch translates: "The sacred cat observes what your model cannot see - that humidity alone tells only part of the story. Winter humidity affects doors differently than summer humidity. The wood remembers the season, even if your measurements do not. Can your wisdom adapt to these deeper patterns?"*

**NEW CONCEPTS:** Feature engineering, seasonal patterns, model limitations  
**DIFFICULTY:** +45% (still Dan 1, but thinking beyond single variables)

In [ ]:
def generate_seasonal_door_data(n_observations: int = 400):
    """
    Generate door data that includes seasonal effects.
    
    Returns:
        Enhanced data with seasonal patterns that pure humidity can't capture
    """
    # Generate humidity as before
    humidity = torch.rand(n_observations, 1) * 70 + 20
    
    # Add seasonal information (0=winter, 1=spring, 2=summer, 3=autumn)
    seasons = torch.randint(0, 4, (n_observations,))
    
    # Seasonal effects on door behavior
    seasonal_adjustments = torch.zeros(n_observations)
    seasonal_adjustments[seasons == 0] = -10  # Winter: doors shrink, stick less
    seasonal_adjustments[seasons == 1] = +5   # Spring: moderate swelling
    seasonal_adjustments[seasons == 2] = +15  # Summer: maximum expansion
    seasonal_adjustments[seasons == 3] = +0   # Autumn: neutral
    
    # Effective humidity includes seasonal wood behavior
    effective_humidity = humidity.squeeze() + seasonal_adjustments
    effective_humidity = torch.clamp(effective_humidity, 0, 100)
    
    # Generate door behavior based on effective humidity
    stick_probability = torch.sigmoid((effective_humidity - 60) / 5.0)
    door_sticks = torch.bernoulli(stick_probability).unsqueeze(1)
    
    return humidity, door_sticks, seasons

def compare_seasonal_vs_simple_models(humidity, door_sticks, seasons):
    """
    Train two models: one with just humidity, one with seasonal awareness.
    
    Returns:
        Comparison of model performances
    """
    # Simple humidity-only model (what you've been building)
    simple_model = DoorStickinessPredictor(input_features=1)
    
    # TODO: Create a seasonal-aware model
    # Hint: You could create features like "summer_humidity" = humidity * is_summer
    # Or create separate humidity thresholds for each season
    
    # TODO: Train both models and compare their performance
    
    # TODO: Calculate accuracy for each season separately
    
    return {}

def analyze_model_limitations(simple_model, humidity, door_sticks, seasons):
    """
    Understand where your humidity-only model fails.
    
    Returns:
        Analysis of prediction errors by season
    """
    with torch.no_grad():
        predictions = simple_model(humidity)
        binary_predictions = (predictions > 0.5).float()
        
        # Calculate accuracy for each season
        seasonal_accuracy = {}
        for season in [0, 1, 2, 3]:
            season_mask = seasons == season
            if torch.sum(season_mask) > 0:
                season_accuracy[season] = torch.mean(
                    (binary_predictions[season_mask] == door_sticks[season_mask]).float()
                ).item()
        
        return seasonal_accuracy

def visualize_seasonal_patterns(humidity, door_sticks, seasons, model=None):
    """Show how door behavior varies by season."""
    season_names = ['Winter', 'Spring', 'Summer', 'Autumn']
    colors = ['blue', 'green', 'red', 'orange']
    
    plt.figure(figsize=(16, 10))
    
    # Seasonal scatter plots
    for i, season in enumerate([0, 1, 2, 3]):
        plt.subplot(2, 4, i+1)
        season_mask = seasons == season
        
        season_humidity = humidity[season_mask]
        season_doors = door_sticks[season_mask]
        
        sticky_mask = season_doors.squeeze() == 1
        smooth_mask = season_doors.squeeze() == 0
        
        plt.scatter(season_humidity[sticky_mask], [1]*torch.sum(sticky_mask).item(),
                   alpha=0.6, color='red', s=30, label='Sticky')
        plt.scatter(season_humidity[smooth_mask], [0]*torch.sum(smooth_mask).item(),
                   alpha=0.6, color='green', s=30, label='Smooth')
        
        plt.title(f'{season_names[i]}')
        plt.xlabel('Humidity (%)')
        plt.ylabel('Door Behavior')
        plt.ylim(-0.2, 1.2)
        plt.yticks([0, 1], ['Smooth', 'Sticky'])
        if i == 0:
            plt.legend()
    
    # Model predictions by season (if provided)
    if model is not None:
        for i, season in enumerate([0, 1, 2, 3]):
            plt.subplot(2, 4, i+5)
            season_mask = seasons == season
            
            season_humidity = humidity[season_mask]
            sorted_indices = torch.argsort(season_humidity.squeeze())
            sorted_humidity = season_humidity[sorted_indices]
            
            with torch.no_grad():
                sorted_predictions = model(sorted_humidity)
            
            plt.plot(sorted_humidity.numpy(), sorted_predictions.numpy(), 
                    color=colors[i], linewidth=2)
            plt.title(f'{season_names[i]} Predictions')
            plt.xlabel('Humidity (%)')
            plt.ylabel('Stick Probability')
            plt.ylim(0, 1)
            plt.axhline(y=0.5, color='purple', linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.show()

# TRIAL: Discover the limitations of your humidity-only model
# SUCCESS: Identify that some seasons are harder to predict than others
# MASTERY: Understand that good models know their limitations
# ENLIGHTENMENT: Realize that feature engineering can be more powerful than complex models

## 🔥 CORRECTING YOUR FORM: A STANCE IMBALANCE

*Cook Oh-Pai-Timizer pauses in their soup preparation, wooden spoon raised thoughtfully*

*"Ah, young grasshopper, I see you rush ahead like boiling water - eager but lacking patience! Your classification stance has become unsteady. See how your sigmoid predictions dance wildly, never settling into the smooth curve they should follow?"*

*A previous student left this flawed door prediction ritual. The training process has lost its balance - can you restore proper technique?*

In [ ]:
def unsteady_classification_training(model, features, target, epochs=2000):
    """This classification training has lost its balance - your form needs correction! 🥋"""
    
    # Using the wrong loss function for classification!
    criterion = nn.MSELoss()  # This is for regression, not classification!
    optimizer = optim.SGD(model.parameters(), lr=0.1)
    
    losses = []
    
    for epoch in range(epochs):
        # Forward pass
        predictions = model(features)
        loss = criterion(predictions, target)
        
        # Backward pass - but missing a critical step!
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
        
        if epoch % 200 == 0:
            print(f'Epoch {epoch}: Loss = {loss.item():.4f}')
    
    return losses

# DEBUGGING CHALLENGE: Can you spot the TWO critical errors in this training ritual?
# 
# HINT 1: What loss function should you use for binary classification?
# Cook's Wisdom: "When predicting yes or no, don't measure with a ruler meant for 'how much'!"
# 
# HINT 2: What essential step is missing between loss.backward() and optimizer.step()?
# Cook's Wisdom: "Clean your bowls before cooking the next dish, or flavors will mix in chaos!"
#
# MASTER'S DEEPER WISDOM: "The eager student uses regression loss for classification problems,
# thinking all prediction is the same. But classification seeks probability, while regression 
# seeks exact values. Choose your loss as carefully as you choose your cooking temperature!"

## 🎊 KATA COMPLETION CERTIFICATE

*Upon successful completion of all trials, Cook Oh-Pai-Timizer approaches with a ceremonial ladle*

**"Congratulations, Grasshopper! You have mastered the Sigmoid Gateway Mystery!"**

### Your Earned Wisdom:
- ✅ **Binary Classification Mastery**: You can now predict yes/no outcomes with neural networks
- ✅ **Sigmoid Activation Understanding**: You transform raw outputs into meaningful probabilities
- ✅ **Decision Threshold Wisdom**: You know how to convert predictions into actionable decisions
- ✅ **Binary Cross-Entropy Fluency**: You measure classification errors with the proper loss function
- ✅ **Confidence Interpretation**: You understand what probability predictions really mean
- ✅ **Data Quality Awareness**: You can handle messy real-world data with preprocessing

### Temple Standing:
**Dan 1 (Temple Sweeper) - Sigmoid Gateway Specialist** 🚪✨

*"You have learned that the sigmoid function is like the temple gate - it stands between chaos and order, transforming any input into the sacred realm of probability. This wisdom will serve you well in your journey toward greater mastery!"*

**Ready for your next challenge? Consider advancing to Dan 2 or exploring more Dan 1 techniques!**